In [3]:
import numpy as np

In [4]:
M = np.array([[0, 1, 1, 1], [1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 0]])
sums = np.sum(M, axis=-1, keepdims=True)
m = np.round(M / (sums + 0.01), 2)
p = m.T
p

array([[0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 1, 0]])

In [8]:
v = np.repeat(0.25, 4).reshape(4, 1)
v

array([[0.25],
       [0.25],
       [0.25],
       [0.25]])

In [9]:
old_v = v
iters = 0
while iters < 1000:
    iters += 1
    new_v = np.matmul(p, old_v)
    p = np.matmul(p, p)
    temp = old_v
    old_v = new_v
    
    if np.sqrt(np.power(temp - new_v, 2).sum()) < 0.02:
        break

In [10]:
old_v

array([[9.70374401e-06],
       [1.41086538e-05],
       [1.41086538e-05],
       [1.41086538e-05]])

In [11]:
np.sqrt(np.power(old_v - v, 2).sum())

0.4999739851618181

In [16]:
def gene_data(data):
    """
    生成user_item 矩阵的 二部图，利用字典存储起来
    """
    data_dict = {}
    m, n = data.shape
    for i in range(m):
        temp_dict = {}
        for j in range(n):
            if data[i, j] != 0:
                temp_dict["d_" + str(j)] = data[i, j]
        data_dict["u_" + str(i)] = temp_dict
        
    for j in range(n):
        temp_dict = {}
        for i in range(m):
            if data[i, j] != 0:
                temp_dict["u_" + str(i)]  = data[i, j]
        data_dict['d_' + str(j)] = temp_dict
    
    return data_dict

# tx = gene_data(M)

{'d_0': {'u_1': 1},
 'd_1': {'u_0': 1, 'u_3': 1},
 'd_2': {'u_0': 1, 'u_3': 1},
 'd_3': {'u_0': 1, 'u_1': 1},
 'u_0': {'d_1': 1, 'd_2': 1, 'd_3': 1},
 'u_1': {'d_0': 1, 'd_3': 1},
 'u_2': {},
 'u_3': {'d_1': 1, 'd_2': 1}}

In [45]:
def update(data, user="u_1", alpha=0.85, maxiters=100, yuzhi=0.001):
    # 将旧的存储起来
    rank = {}
    for k in data.keys():
        rank[k] = 0
    rank[user] = 1
    
    maxs = 0
    while maxs < maxiters:
        temp = {}
        for k in data.keys():
            temp[k] = 0  
        maxs += 1
        for key, value in data.items():
            for k, v in value.items():
                temp[k] += alpha * rank[key] / (len(value) + 0.01)
                if k == user:
                    temp[k] += (1 - alpha)         
        sums = 0
        for k in data.keys():
            sums += temp[k] - rank[k]  
        if abs(sums) < yuzhi:
            break
        rank = temp
        if maxs % 10 ==0:
            print("maxs {}, sums={}".format(maxs, sums))     
    return rank

In [43]:
def recomd(tx, user, top_k=2):
    scores = update(tx, user=user)
    
    # 计算user 已经购买过的 item
    items = tx[user]
    
    orl = []
    for k in items.keys():
        orl.append(k)
        
    remcomds = {}
    for k in scores.keys():
        if k.startswith('d_'):
            if k not in orl:
                remcomds[k] = scores[k]
                
    remcomds = sorted(remcomds.items(), key=lambda row: row[1], reverse=True)
    if len(remcomds) < 2:
        return remcomds
    else:
        return remcomds[:2]
        
recomd(tx, user="u_0")

maxs 10, sums=0.06559505529113482
maxs 20, sums=0.012287146611602878
maxs 30, sums=0.00230147841537838


[('d_0', 0.10488655364125173)]